In [ ]:
import ctypes
import numpy as np
import time
import cv2
import gc
import os

core = os.path.abspath("./pcsx2_libretro.dll")
game = os.path.abspath("D:/Downloads/Granturismo3.iso")

lib = ctypes.CDLL("./pcsx2_headless.dll")
     
lib.init.argtypes = [ctypes.c_char_p, ctypes.c_char_p]
lib.init.restype = ctypes.c_void_p

lib.get_frame.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_uint8), 
    ctypes.c_int, 
    ctypes.c_int
]

# lib.run.argtypes = [ctypes.c_void_p]
lib.run.restype = None

core_bytes = core.encode('utf-8')
game_bytes = game.encode('utf-8')

lib.init(core_bytes, game_bytes)

lib.get_frame.argtypes = [
    ctypes.POINTER(ctypes.c_uint8), 
    ctypes.c_int, 
    ctypes.c_int
]

gc.collect()


def get_frame(width=640, height=480):
    buffer = (ctypes.c_uint8 * (width * height * 3))()
    lib.get_frame(buffer, width, height)
    
    img = np.frombuffer(buffer, dtype=np.uint8)
    return img.reshape((height, width, 3))[::-1]

while True:
    start = time.time_ns()
    lib.run()
    img = get_frame()
    print(time.time_ns() - start)

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    cv2.imshow("", img)
    cv2.waitKey(1)

2001000
85958400
1003900
1001200
1001700
999400
1151600
997700
1011800
997700
0
0
0
0
0
0
0
1002000
1003900
0
0
1002400
10008100
994400
1999900
1000700
2998000
1001300
2016500
1000900
2001900
1004900
2002000
1002900
2001900
1001200
2006500
1511500
5004100
1006400
2002600
2003500
1997800
1000600
2000600
1005200
2004200
1007000
1999500
1999800
1999800
1002900
2000800
1004200
2005200
2005400
6005200
1998900
2003000
1999000
2001300
1000000
2002500
1999900
2003900
1003200
1001000
1001400
2001900
999100
1351600
1002500
2002500
1018300
2005000
1001300
2001700
1000900
2001000
999400
1002600
1019500
2000800
999900
2004800
1004400
2002000
3000800
2000700
1002400
1006600
1017400
2000400
1001100
1998800
1001400
2001300
1001000
1999500
1005900
1999800
997800
2021500
1001400
2001200
1004900
1999800
1002600
1999800
1001300
2000700
1002300
2002100
2000600
1998600
1000100
1999200
999700
2014300
1000100
1018500
1002000
2002000
1017100
2004400
1016800
2000200
1001900
1019300
1001100
1005500
997700
200220